### Lab Tasks
- In the dataframe creates in Lab 2 - Part a set ``Salary`` as the target value. 
- The rest of the columns are considered as X, feature set. 
- Use ``train_test_split`` to split the dataset into train and test dataset. set ``random_state = 0``.
- Use ``MinMaxScaler`` to scale feature set X. 

### Data Preprocessing

In [1]:
import pandas as pd
data = pd.read_csv('adult.csv')

In [2]:
data = data[data != ' ?']

C:\Users\obaid\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [3]:
l = ['workclass', 'education', 'occupation', 'native-country']
data.drop(l, axis = 1, inplace = True)

In [4]:
df = pd.get_dummies(data['marital-status'])
data = pd.concat([data, df], axis = 1)
data.drop('marital-status', axis = 1, inplace=True)

In [5]:
df = pd.get_dummies(data['relationship'])
data = pd.concat([data, df], axis = 1)
data.drop('relationship', axis = 1, inplace=True)

In [6]:
df = pd.get_dummies(data['race'])
data = pd.concat([data, df], axis = 1)
data.drop('race', axis = 1, inplace=True)

In [7]:
data['sex'] = data['sex'].map({' Male':0, ' Female':1})

In [8]:
data['Salary'] = data['Salary'].map({' <=50K':0, ' >50K':1})

In [9]:
y = data['Salary']
X = data.drop(['Salary'], axis = 1)

In [10]:
X.shape

(32561, 25)

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

x_train_org,x_test_org,y_train,y_test=train_test_split(X,y,random_state=0)  ## org stands for the very original
scaler = MinMaxScaler()
X_train = scaler.fit_transform(x_train_org)  ### you can fit and transform together
X_test = scaler.transform(x_test_org) 

C:\Users\obaid\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [12]:
### Lets do PCA ### It is done after preprocessing and split of data

from sklearn.decomposition import PCA

pca = PCA(n_components= 0.95)

X_train = pca.fit_transform(X_train)

X_test = pca.transform(X_test)

#### Question 1
Use PCA is to reduce the size of the feature space while retaining 95% of explained variance. What is the size of the transformed feature space? 

In [13]:
### X is reduced from this point onwards 
print(X_train.shape)
(pca.n_components_)

(24420, 12)


12

#### Question 2
For the rest of this lab, we will consider reduced train and test sets. For models with hyper-parameter ``random_state = 0``. 

Use grid search to find the best parameters of a support vector machine with kernel ``'rbf'`` and following range of parameters: 
```Python
C in [0.1, 1, 10]
gamma in [0.1, 1, 10]
cv = 5```

What are the best parameters of this model? 

In [14]:
# lets build a SVM kernel rbf model  and then we will feed it to gridsearch
from sklearn.model_selection import GridSearchCV
from  sklearn.svm import SVC

param_grid = { 'C':[0.1,1,10],
             'gamma':[0.1,1,10],
             'kernel': ['rbf'],
             'random_state' :[0]}


grid_search = GridSearchCV(SVC(),param_grid, cv=5 )

grid_search.fit(X_train,y_train)

print("best paramaters are:{}".format(grid_search.best_params_))
print("best accuracy score: {:.3f}".format(grid_search.best_score_))

### elearning is giving the answer C = 10 gamma = 0.1

best paramaters are:{'C': 10, 'gamma': 10, 'kernel': 'rbf', 'random_state': 0}
best accuracy score: 0.827


In [15]:
grid_search.best_estimator_

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=10, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

#### Question 3
Train a gradient boosting machine learning model on this dataset with the following hyper-parameters. 
```Python
n_estimators= 100
learning_rate=0.5
random_state = 0
```

What is the auc score of test set? (2 significant digits)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state=0, n_estimators = 100, learning_rate =0.5)
gbrt.fit(X_train, y_train)

pred = gbrt.predict(X_test)
pred = pd.Series(pred)

print("Accuracy on training set: {:.3f}".format(gbrt.score(X_train, y_train)))
print("Accuracy on testing  set: {:.3f}".format(gbrt.score(X_test, y_test)))
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)

metrics.auc(fpr, tpr)

Accuracy on training set: 0.863
Accuracy on testing  set: 0.820


0.7350312989006923

In [17]:
y_test.shape

(8141,)

In [18]:
type(pred)

pandas.core.series.Series

#### Question 4

Using the model created for previous questions, call ``predict_proba`` on the test set. Now change the threshold from 0.5 to 0.75, which means if the probability of an instance belongs to class 0 is higher than or equal to 0.75, then the prediction is label 0, otherwise is label 1. 

Now, what is the auc score of test dataset? (2 significant digits)

**Note:** The first index of ``predict_proba`` refers to the probability that the data belong to class 0, and the second refers to the probability that the data belong to class 1.

In [19]:
thresh=0.75
pred_df = pd.DataFrame( gbrt.predict_proba(X_test) )

In [20]:
pred_df['pred'] =  pred_df[0].apply(lambda x: 0 if  x>=thresh else 1)

In [21]:
pred_df.head(5)

,0,1,pred
0,0.983631,0.016369,0
1,0.975435,0.024565,0
2,0.824839,0.175161,0
3,0.973031,0.026969,0
4,0.664415,0.335585,1


In [22]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_df['pred'])
metrics.auc(fpr, tpr)

0.7973661803446476